In [1]:
# import packages
import tensorflow as tf
import matplotlib.image as mpimg
import cv2
import numpy as np 
import time
import argparse
import imutils
import dlib
from imutils.video import VideoStream
import pandas as pd

In [2]:
class TrafficLightClassifier(object):
    def __init__(self):
        PATH_TO_MODEL = './export_pb_graph/frozen_inference_graph.pb'
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            # Works up to here.
            with tf.gfile.GFile(PATH_TO_MODEL, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
            self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
            self.d_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
            self.d_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
            self.d_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
            self.num_d = self.detection_graph.get_tensor_by_name('num_detections:0')
        self.sess = tf.Session(graph=self.detection_graph)
    
    
    def get_classification(self, img):
        # Bounding Box Detection.
        with self.detection_graph.as_default():
            # Expand dimension since the model expects image to have shape [1, None, None, 3].
            img_expanded = np.expand_dims(img, axis=0)  
            (boxes, scores, classes, num) = self.sess.run(
                [self.d_boxes, self.d_scores, self.d_classes, self.num_d],
                feed_dict={self.image_tensor: img_expanded})
        return boxes, scores, classes, num

In [3]:
class_id_to_name = {'1': 'person'}

In [4]:
person = TrafficLightClassifier()

In [5]:
count = 0

In [6]:
vs = VideoStream(src=0).start()
time.sleep(3.0)
flag = 1

while True:
    if flag == -1:
        img = vs.read()
        #img = cv2.resize(img, (700, 700))
        rboxes, rscores, rclasses, rnum = person.get_classification(img[:,:,::-1])
        h = img.shape[0]
        w = img.shape[1]
        for i in range(rboxes.shape[1]):
            if rscores[0][i] > 0.5:
                sx = int(rboxes[0][i][1] * w)
                sy = int(rboxes[0][i][0] * h)
                ex = int(rboxes[0][i][3] * w)
                ey = int(rboxes[0][i][2] * h)
                key = str(int(rclasses[0][i]))
                name = class_id_to_name[key]
                cv2.rectangle(img, (sx, sy), (ex, ey), (0, 0, 255), 2)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(img, name, (sx-5, ey + 26), font, 0.8, (0, 0, 255), 1)
                
                timestamp = time.time()
                count = count + 1
                if count % 50 == 0:
                    cv2.imwrite('./save_images/' + str(timestamp) + '.jpg', img)
        
        cv2.imshow('finger', img)
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord("q"):
            break
    flag = flag * -1

cv2.destroyAllWindows()
vs.stop()

In [13]:
img_path = '/home/mao/Pictures/test_image/000002.jpg'

In [14]:
image = mpimg.imread(img_path)

In [15]:
rboxes, rscores, rclasses, rnum = person.get_classification(image)

In [16]:
h = image.shape[0]
w = image.shape[1]

In [17]:
cv2_image = cv2.imread(img_path)

In [18]:
for i in range(rboxes.shape[1]):
    if rscores[0][i] > 0.3:
        sx = int(rboxes[0][i][1] * w)
        sy = int(rboxes[0][i][0] * h)
        ex = int(rboxes[0][i][3] * w)
        ey = int(rboxes[0][i][2] * h)
        key = str(int(rclasses[0][i]))
        name = class_id_to_name[key]
    cv2.rectangle(cv2_image, (sx, sy), (ex, ey), (0, 0, 255), 2)
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(cv2_image, name, (sx-5, ey + 26), font, 0.8, (0, 0, 255), 1)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

In [19]:
cv2.imshow('person', cv2_image)
cv2.waitKey(0)
cv2.destroyAllWindows()